In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import cv2
import glob
import time
import json
import random
import warnings

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn import tree
from sklearn import impute
from sklearn import metrics
from sklearn import ensemble
from sklearn import linear_model
from sklearn import decomposition
from sklearn import preprocessing
from sklearn import model_selection


warnings.filterwarnings('ignore')

SEED = 42
np.random.seed(SEED)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('float_format', '{:f}'.format)

sns.set_style("white")
mpl.rcParams['figure.dpi'] = 600
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing 
from sklearn import model_selection
from sklearn.preprocessing import OrdinalEncoder 
from sklearn.model_selection import train_test_split 
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
import optuna
from sklearn import linear_model


In [ ]:
df = pd.read_csv("../input/september-folds/train_folds.csv")
df_test=pd.read_csv("../input/tabular-playground-series-sep-2021/test.csv")
sample_submission=pd.read_csv("../input/tabular-playground-series-sep-2021/sample_solution.csv")
df1=pd.read_csv("../input/catboost-1-iit/train_pred_1.csv")
df2=pd.read_csv("../input/lgbm-2-iit/train_pred_2.csv")
df3=pd.read_csv("../input/lgbm-3-iit/train_pred_3.csv")
df4=pd.read_csv("../input/lgbm-4-iit/train_pred_4.csv")
df5=pd.read_csv("../input/lgbm5-iit/train_pred_5.csv")
df6=pd.read_csv("../input/catboost6-iit/train_pred_6.csv")
df7=pd.read_csv("../input/catboost7-iit/train_pred_7.csv")

In [ ]:
df_test1=pd.read_csv("../input/catboost-1-iit/test_pred_1.csv")
df_test2=pd.read_csv("../input/lgbm-2-iit/test_pred_2.csv")
df_test3=pd.read_csv("../input/lgbm-3-iit/test_pred_3.csv")
df_test4=pd.read_csv("../input/lgbm-4-iit/test_pred_4.csv")
df_test5=pd.read_csv("../input/lgbm5-iit/test_pred_5.csv")
df_test6=pd.read_csv("../input/catboost6-iit/test_pred_6.csv")
df_test7=pd.read_csv("../input/catboost7-iit/test_pred_7.csv")

In [ ]:
df=df.merge(df1,on="id",how="left")
df=df.merge(df2,on="id",how="left")
df=df.merge(df3,on="id",how="left")
df=df.merge(df4,on="id",how="left")
df=df.merge(df5,on="id",how="left")
df=df.merge(df6,on="id",how="left")
df=df.merge(df7,on="id",how="left")
df_test =df_test.merge(df_test1,on="id",how="left")
df_test =df_test.merge(df_test2,on="id",how="left")
df_test =df_test.merge(df_test3,on="id",how="left")
df_test =df_test.merge(df_test4,on="id",how="left")
df_test=df_test.merge(df_test5,on="id",how="left")
df_test=df_test.merge(df_test6,on="id",how="left")
df_test=df_test.merge(df_test7,on="id",how="left")

In [ ]:
features =[col for col in df.columns if col not in ["kfold","claim"]]

In [ ]:
df['min_row'] = df[features].min(axis=1)
df['max_row'] = df[features].max(axis=1)
df['mean_row'] = df[features].mean(axis=1)
df['std_row'] = df[features].std(axis=1)
df['median_row']=df[features].median(axis=1)
df_test ['min_row'] = df_test [features].min(axis=1)
df_test ['max_row'] = df_test [features].max(axis=1)
df_test ['mean_row'] = df_test [features].mean(axis=1)
df_test ['std_row'] = df_test [features].std(axis=1)
df_test['median_row']=df_test[features].median(axis=1)


In [ ]:
useful_features=["pred_1","pred_2","pred_3","pred_4","pred_5","pred_6","pred_7","min_row","max_row","mean_row","std_row","median_row"]


In [ ]:
X=df[useful_features]
y=df["claim"]
from sklearn.feature_selection import  SelectKBest 
from sklearn.feature_selection import f_classif 
best_features =SelectKBest(score_func=f_classif,k=12)
fit=best_features.fit(X,y)

In [ ]:
df_scores =pd.DataFrame(fit.scores_)
df_columns =pd.DataFrame(X.columns)

In [ ]:
features_scores =pd.concat([df_columns,df_scores],axis=1)
features_scores.columns=["space","scores"]

In [ ]:
features_scores=features_scores.nlargest(12,"scores")

In [ ]:
features_scores

In [ ]:
useful_features=features_scores["space"]

In [ ]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [ ]:
xgb_params = {
    'n_estimators' : 500,
    'reg_lambda' : 3,
    'reg_alpha' : 26,
    'subsample' : 0.6000000000000001,
    'colsample_bytree' : 0.6000000000000001,
    'max_depth' : 9,
    'min_child_weight' : 5,
    'gamma' : 13.054739572819486,
    'learning_rate': 0.01,
    'booster': 'gbtree'
}

lgbm_params = {
    "objective": "binary",
    "learning_rate": 0.0136989525,
    'n_estimators':595,
    'num_leaves': 99,
    'min_child_samples': 7,
    'feature_fraction': .3107239,
    'bagging_fraction': .62804758961,
    'bagging_freq': 3,
    'reg_alpha': 89.71464915,
    'reg_lambda': 1
}

catb_params = {
    'iterations': 709, 
    'objective': 'CrossEntropy', 
    'bootstrap_type': 'Bernoulli', 
    'od_wait': 1144, 
    'learning_rate': 0.00890385, 
    'reg_lambda': 40.86021936, 
    'random_strength': 58.521645, 
    'depth': 7, 
    'min_data_in_leaf': 6, 
    'leaf_estimation_iterations': 1, 
    'subsample': .100058356,
    'verbose' : 0
}
lgbm1_params = {
    'objective': 'binary',
    'n_estimators': 5,
    'learning_rate': 5e-3,
    'subsample': 0.6,
    'subsample_freq': 1,
    'colsample_bytree': 0.4,
    'reg_alpha': 10.0,
    'reg_lambda': 1e-1,
    'min_child_weight': 256,
    'min_child_samples': 20,
    'importance_type': 'gain',

}
lgbm2_params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'n_estimators': 5, 
    'learning_rate': 0.005, 
    'num_leaves': 29,
    'min_child_samples': 236, 
    'lambda_l1': 7.702002052840491, 
    'lambda_l2': 9.738840335016775, 
    'feature_fraction': 0.05, #0.3811137625854881, 
    'bagging_fraction': 0.7345219542805319, 
    'bagging_freq': 3, 
    'min_child_weight': 280.0714278010327}





Level-1

In [ ]:
#level-1/model-1
final_test_preds=[] 
final_valid_preds={}
score=[]
for fold in range(5):
    xtrain=df[df.kfold !=fold].reset_index(drop=True) 
    xvalid=df[df.kfold ==fold].reset_index(drop=True)
    xtest=df_test.copy()
    valid_ids=xvalid.id.values.tolist()
    ytrain=xtrain['claim']
    yvalid=xvalid['claim']

    xtrain=xtrain[useful_features].to_numpy()
    xvalid=xvalid[useful_features].to_numpy()
    xtest =xtest[useful_features].to_numpy()
    oof_preds, oof_y = [], []
    
    test_preds = np.zeros((xtest.shape[0]))
    model=CatBoostClassifier(**catb_params)
    
    model = model.fit(xtrain, ytrain)
    preds =model.predict_proba(xvalid)
    oof_preds.extend(preds[:, 1])
    oof_y.extend(yvalid)
        
    test_preds += model.predict_proba(xtest)[:, 1]
    final_test_preds.append(test_preds)
    final_valid_preds.update(dict(zip(valid_ids,preds[:,1])))
    rmse = roc_auc_score(yvalid, preds[:, 1])
    
    print(f"ROC AUC of current fold is {rmse}")
        
    rmse = roc_auc_score(oof_y, oof_preds)
    score.append(rmse)
print("SCORE")
print(np.mean(score))

In [ ]:
final_valid_preds =pd.DataFrame.from_dict(final_valid_preds,orient="index").reset_index()
final_valid_preds.columns=["id","pred_1"]
final_valid_preds.to_csv("level1_train_pred_1.csv",index =False)

sample_submission.claim =np.mean(np.column_stack(final_test_preds),axis=1)
sample_submission.columns=["id","pred_1"]
sample_submission.to_csv("level1_test_pred_1.csv",index=False)

In [ ]:
#level-1/model-2
final_test_preds=[] 
final_valid_preds={}
score=[]
for fold in range(5):
    xtrain=df[df.kfold !=fold].reset_index(drop=True) 
    xvalid=df[df.kfold ==fold].reset_index(drop=True)
    xtest=df_test.copy()
    valid_ids=xvalid.id.values.tolist()
    ytrain=xtrain['claim']
    yvalid=xvalid['claim']

    xtrain=xtrain[useful_features].to_numpy()
    xvalid=xvalid[useful_features].to_numpy()
    xtest =xtest[useful_features].to_numpy()
    oof_preds, oof_y = [], []
    
    test_preds = np.zeros((xtest.shape[0]))
    model=LGBMClassifier(**lgbm_params)
    
    model = model.fit(xtrain, ytrain)
    preds =model.predict_proba(xvalid)
    oof_preds.extend(preds[:, 1])
    oof_y.extend(yvalid)
        
    test_preds += model.predict_proba(xtest)[:, 1]
    final_test_preds.append(test_preds)
    final_valid_preds.update(dict(zip(valid_ids,preds[:,1])))
    rmse = roc_auc_score(yvalid, preds[:, 1])
    
    print(f"ROC AUC of current fold is {rmse}")
        
    rmse = roc_auc_score(oof_y, oof_preds)
    
    score.append(rmse)
print("SCORE")
print(np.mean(score))

In [ ]:
final_valid_preds =pd.DataFrame.from_dict(final_valid_preds,orient="index").reset_index()
final_valid_preds.columns=["id","pred_2"]
final_valid_preds.to_csv("level1_train_pred_2.csv",index =False)

sample_submission.claim =np.mean(np.column_stack(final_test_preds),axis=1)
sample_submission.columns=["id","pred_2"]
sample_submission.to_csv("level1_test_pred_2.csv",index=False)

In [ ]:
#level-1/model-3
final_test_preds=[] 
final_valid_preds={}
score=[]
for fold in range(5):
    xtrain=df[df.kfold !=fold].reset_index(drop=True) 
    xvalid=df[df.kfold ==fold].reset_index(drop=True)
    xtest=df_test.copy()
    valid_ids=xvalid.id.values.tolist()
    ytrain=xtrain['claim']
    yvalid=xvalid['claim']

    xtrain=xtrain[useful_features].to_numpy()
    xvalid=xvalid[useful_features].to_numpy()
    xtest =xtest[useful_features].to_numpy()
    oof_preds, oof_y = [], []
    
    test_preds = np.zeros((xtest.shape[0]))
    model=XGBClassifier(**xgb_params)
    
    model = model.fit(xtrain, ytrain)
    preds =model.predict_proba(xvalid)
    oof_preds.extend(preds[:, 1])
    oof_y.extend(yvalid)
        
    test_preds += model.predict_proba(xtest)[:, 1]
    final_test_preds.append(test_preds)
    final_valid_preds.update(dict(zip(valid_ids,preds[:,1])))
    rmse = roc_auc_score(yvalid, preds[:, 1])
    
    print(f"ROC AUC of current fold is {rmse}")
        
    rmse = roc_auc_score(oof_y, oof_preds)
    
    score.append(rmse)
print("SCORE")
print(np.mean(score))

In [ ]:
final_valid_preds =pd.DataFrame.from_dict(final_valid_preds,orient="index").reset_index()
final_valid_preds.columns=["id","pred_3"]
final_valid_preds.to_csv("level1_train_pred_3.csv",index =False)

sample_submission.claim =np.mean(np.column_stack(final_test_preds),axis=1)
sample_submission.columns=["id","pred_3"]
sample_submission.to_csv("level1_test_pred_3.csv",index=False)

In [ ]:
df = pd.read_csv("../input/september-folds/train_folds.csv")
df_test=pd.read_csv("../input/tabular-playground-series-sep-2021/test.csv")
sample_submission=pd.read_csv("../input/tabular-playground-series-sep-2021/sample_solution.csv")

df1=pd.read_csv("level1_train_pred_1.csv")
df2=pd.read_csv("level1_train_pred_2.csv")
df3=pd.read_csv("level1_train_pred_3.csv")


df_test1=pd.read_csv("level1_test_pred_1.csv")
df_test2 =pd.read_csv("level1_test_pred_2.csv")
df_test3=pd.read_csv("level1_test_pred_3.csv")

In [ ]:
df=df.merge(df1,on="id",how="left")
df=df.merge(df2,on="id",how="left")
df=df.merge(df3,on="id",how="left")


In [ ]:
df_test =df_test.merge(df_test1,on="id",how="left")
df_test=df_test.merge(df_test2,on="id",how="left")
df_test =df_test.merge(df_test3,on="id",how="left")

In [ ]:
useful_features=["pred_1","pred_2","pred_3"]

In [ ]:
final_test_preds=[] 
final_valid_preds={}
score=[]
for fold in range(5):
    xtrain=df[df.kfold !=fold].reset_index(drop=True) 
    xvalid=df[df.kfold ==fold].reset_index(drop=True)
    xtest=df_test.copy()
    valid_ids=xvalid.id.values.tolist()
    ytrain=xtrain['claim']
    yvalid=xvalid['claim']

    xtrain=xtrain[useful_features].to_numpy()
    xvalid=xvalid[useful_features].to_numpy()
    xtest =xtest[useful_features].to_numpy()
    oof_preds, oof_y = [], []
    
    test_preds = np.zeros((xtest.shape[0]))
    model=linear_model.LogisticRegression(solver='liblinear',max_iter=10,random_state=42)
    model = model.fit(xtrain, ytrain)
    preds =model.predict_proba(xvalid)
    oof_preds.extend(preds[:, 1])
    oof_y.extend(yvalid)
        
    test_preds += model.predict_proba(xtest)[:, 1]
    final_test_preds.append(test_preds)
    final_valid_preds.update(dict(zip(valid_ids,preds[:,1])))
    rmse = roc_auc_score(yvalid, preds[:, 1])
    
    print(f"ROC AUC of current fold is {rmse}")
        
    rmse = roc_auc_score(oof_y, oof_preds)
    
    score.append(rmse)
print("SCORE")
print(np.mean(score))

In [ ]:
sample_submission.claim =np.mean(np.column_stack(final_test_preds),axis=1)
sample_submission.to_csv("submission.csv",index =False)